## Intro
### Contribution
1. propose a TEE and blockchain assisted cross domain authorization system.
2. abac based dynamic iot cross-domain authorization (ABAC is dynamic because access decisions are evaluated at runtime based on current attribute values, not static role assignments.)
3. experiment to evaluate proposed system's performance
4. avoid leak metadata of , and access logic
5. Traceability access control


#### Scalability in Large-Scale IoT Systems

In IoT environments:
- devices are numerous
- identities are dynamic
- cross-domain access is common

IBS requires:
- tight coupling between identity and access rights
- frequent key updates when permissions change

ABAC decouples:
- identity management
- access policies

This allows policies to evolve without reissuing identities or keys, significantly improving scalability.

#### Cross-Domain Authorization Support

IBS-based systems assume a single trusted Private Key Generator (PKG) or tightly coordinated PKGs across domains.
Problems:
- PKG trust becomes a bottleneck
- cross-domain trust establishment is complex
- key escrow issues arise

ABAC enables:
- attribute verification across domains
- policy enforcement without shared identity infrastructure
- blockchain-backed attribute provenance

This aligns naturally with your cross-domain fog–blockchain architecture.

#### Reduced Key Management Complexity

IBS introduces:
- key escrow risk (PKG knows private keys)
- heavy cryptographic operations (pairings)
- complex revocation mechanisms

ABAC:
- uses standard public-key authentication for identity
- assigns attributes independently
- supports policy updates without rekeying

This is more suitable for resource-constrained IoT devices.

#### Security Policy Evolution

IoT systems evolve:
- devices change roles
- policies change frequently
- context changes dynamically

With IBS:
- permission changes require identity-level changes
- revocation is coarse-grained

With ABAC:
- attributes can be updated or revoked
- policies evolve independently
- authorization remains deterministic and auditable

#### Compatibility with TEE-Based Enforcement

ABAC policy evaluation:
- is deterministic
- operates on structured inputs
- fits well inside TEE execution

IBS:
- only verifies cryptographic signatures
- cannot express or enforce policies inside the enclave

Thus, ABAC integrates naturally with your TEE-based authorization design.

## Background


---
### Attribute Based Access Control Mechanism
The ABAC mechanism is a flexible and scalable model that supports real-time and fine-gained access control. Prior studies have shown that ABAC is an optimal choice for IoT systems [][], since it can comprehensively accommodate IoT system's sophisticated environments,heterogeneous devices, and numerous dynamic access requirements.

The foundation of ABAC is the concept of attributes, which describe entities, resources, and conditions involved in access requests. By customizing access policy over these attributes, ABAC is able to process numerous access requests from a large number of subject 
ABAC mechnaism is consisted of 4 fields:(subject attribute, object attribute, enviroment attribute and policy)
1. **subject attribute** describes the entity requesting access, such as identity, role, department, and device capabilities, which indicate whether the entity is qualified to access target resource.
2. **object attribute** describes the properties of resources, such as resource type, ownership, and sensitity level, which 
3. **environment attribute** describes contextual information that is independent of both subjects and objects, such as time, network state, or system workload. These attributes assist the policy to making context-aware authorization decisions.
4. **policy** define the access standard based on the combination of above attributes, and will output a boolean value that 1 means permit and 0 means denied.

In our proposed system, because the IoT device may be the resource provider and consumer at same time, so it may contain both subject and object attributes.



---
### Blockchain
Blockchain is a type of distributed ledger system that designed to maintain a shared and consistent record of transactions across a Peer-to-Peer(P2P) network. Compared to relying on centralized authorities, participants collectively validate uploaded transactions through consensus protocols to enhanse data integrity.
A blockchian is constructed as an ordered sequence of blocks which contain a set of validated transcations. Each block's header cryptographicly stores the hash of pervious block, as shown in Fig, forming an chain structure. The attacker wishs to manipulate data stored on the blockchain would be required to recompute all subsequent blocks and control the majority of the participating nodes to achieve consensus. Such action is difficult and costly, thus blockchain can be utilized as a tamper-resistant and verifiable databased for proposed system.

In proposed system, since not all enetities are jointed to the blockchain, so we used a permissioned blockchain. Compared to the permissionless blockchain, which open to join for any enetities, a permissioned blockchain is constructed by authenticated participants. This approach also reduces consensus overhead and improves throughput.

Another service provides by blockchain is smart contract which is the automatautomatically executeded deterministic programs. 



---
### Trusted Execution Environment  
However permissioned is not equal to trusted. Even in a permissioned setting, participants may be compromised or exist insider attacks. To enhance confidentiality and execution integrity, TEEs are employed.  


TEE helps to fill the security gap on-chian database and off-chain execution. The TEE is a secure zone within the processor named enclave. The enclave provides the exectuion's confidentiality and integrity even if the host is operating-level compromised. 

TEEAttestation(SK_TEE, output) -> p, like a TEE's signature on output.  
VerifyAttestation(PK_TEE, output, p) -> 1/0 : the output is generated by TEE or not.







## Architecture  


---
### Components
1. Fog node: They are (Fog Nodes Are Blockchain Validators / Peers, which Propose transactions (registration, revocation); Validate transactions from other fog nodes; Reach consensus on block ordering; Store replicated ledger copies) partially trusted, means they may be OS-level compromised or exist malicious insiders, but the Trusted Execution Environment on each fog node is assumed to be secure and provides hardware-enforced isolation. Fog nodes in each domain jointly maintain a blockchain and manage Iot devices in the domain. It deploys TEE and runs smart contracts in the enclave. It can be treated as a Attribute Authority in the ABAC system. It also store registered iots' info(attributes/access policies) securitly and back up those info to cloud.
2. Iot node: They play the role of resource providers or consumers or both in the domain. Compared to the Fog node, IoT’s computing and storage capacity is relatively weaker and untrusted. It may ask access to other devices or resources inside or outside its own domain, which requires authorization from the blockchain maintained by Fog nodes.
3. DDS: The cloud-based Dedicated Data Storage (DDS) acts as a trusted, robust backend storage service. Unlike traditional centralized authorization servers, the cloud/DDS does not participate in real-time authentication or authorization decisions. Instead, it provides durability, recoverability, and global availability for authorization-related data. In addition to serving as a trusted storage backend, the cloud can also act as a resource provider(application services, analytics, historical data, resources that IoT devices may want to access). Access to cloud-hosted resources is also authorized by fog nodes, while the cloud enforces access by verifying short-lived authorization tokens without participating in policy evaluation.
4. blockchain: Serve as a global distributed ledger, the blockchain in this system carries the information of different domains which will be used for cross-domain authentication. The registered IoTs' identity information, as well as revoking information, will be stored on the blockchain for later authentication.

5. Attribute Authority: a trust part of fog nodes, which is controlled by system manager or administrator. They contain the table of deviceID and corresponding attributes generated based on information provided by device manufacturers. Meanwhile, they also generate and maintain the access policies, which will be stored on-chain as smart contracts.



---


### Attributes defination
1. Subject attributes
- role: employee, visitor, security staff, admin...
- Organization:
- department: HR, Engineering, Maintenance...
- Office: 
- deviceType: sensor, actuator, gateway, monitor...
- location: floor number or zone

2. Object attributes
- resourceType: temperatureSensor, doorLock, camera, HVAC, storage...
- OwnerDept: Engineering, Finical
- sensitivityLevel: low, medium, high
- location: same zone/floor

3. Environmental attributes
- time: working hours or not
- emergency_mode: Boolean
- system_load: 80%

ACTION: READ; WRITE; EXCUTE.


### Policy defination


permit if
  subject.role = "employee" AND
  object.resourceType = "doorLock" AND
  subject.location = object.location AND
  environment.time ∈ working_hours AND
  ACTION = "EXECUTE"

permit if
  subject.dep = "security" AND
  subject.deviceType = "computer" AND
  object.resourceType = "monitor_data" AND
  environment.time ∈ working_hours AND
  ACTION = "READ"


access request=(SubID, ObjID, Action, Environment)

SELECT policy_id  
WHERE Obj.resource_type = "SENSOR_DATA"  
AND Sub.dep = "Guard"
AND action = "READ"  
AND status = "ACTIVE"  

policies are stored as a list in smart contract


### Security model

#### Adversary model
The proposed system consider adversaries with following capabilities:
1. the adversart can be iot devices which try ti gain unauthorized access to system resources.
2. it may compromised the fog node or be inside attacker which gain the OS-level of control of fog node, in order to manipulate the access process or obtain secure information in system, such as attributes or policy's conditions.

#### Trust Assumption
1. blockchain and TEE are relible, means 
2. DDS is roubut for data confidentiality, integrity, and availability.
3. the device attribute table maintained by AA is assume to be reliable, trusted.

#### Design Goals
1. Prevent illegal access from compromised or malicious IoT devices without satisfying ABAC policies.
2. prevent tampering the attribute or policy by adversaries.
3. even the fog node failed by accident, registered io devices's registered state and attributes wouldn't loss, and can recover from blockchain and DDS.



### Workflow

---

#### Registration

In the system, manufacturers of IoT devices are expected to provide the information of a device's Device_id to respect functionality and capability. Attribute Authorities(AAs) collect these kind of info from manufacturers abd form a Device-Attribute Mapping Table:
(N/A means this device doesn't consume or provide resource)


| Device-id                   | Subject Attribute         | Object Attribute       | Manufacturer        |
| --------------------------- | ------------------------- | -----------------------| ------------------- |
| x0x....                     | N/A                       | ......                 | Company A           |
| x0b....                     | .......                   | N/A                    | Company A           |
| abx....                     | .......                   | N/A                    | Company X           |
| NVx....                     | .......                   | ......                 | Company N           |

device-id != uid  
device-id for attribute authority  
UID is for identity in proposed system.


Step 1 the IoT device sends request message Enc_AA(device_ID, Manufacturer_Meta) that encrypted by AA's public key to the closest AA for it. 

Step 2 Once receiving the message, AA will check the device_ID according to manufacturer providing information, the Manufacturer_Meta assists to verify the intergrity of device_ID.
After that, AA fetches corresponding attributes(subject and object which exist) from Device-Attribute Mapping Table, and policies P[policy_IDs: policies] contain conditions of those attributes. AA will put above data with IoT's UID into message m and sign on it, then send encrypted Enc_TEE(m, sign) to FN's TEE in the IoT's domain.

Step 3 The TEE decrypt message with TEE_private key, and verify signature with SignVerify(m, sign, AA_public key). After that, TEE seperately seal atributes and policyies with AES and store sealed data with bind UID in FN's normal space(outside enclave). TEE will sign on message m=(UID,FN_id) to form a response r=(m,sign), then send response to the IoT and transaction contain this response to blockchain. 

step 4. The blockchain verify the TEE signature then records the (UID, FN_id) binding in the IoT–Fog mapping table, indicating the fog domain responsible for the registered IoT device.



System Assumptions

- Participating IoT devices are manufactured by trusted manufacturers.  
- Each device is provisioned during manufacturing with a device identifier bound to a manufacturer-issued cryptographic credential, which serves as the root of trust (RoT) for device authentication.  
- The Attribute Authority (AA) maintains a Device–Attribute Mapping Table, populated using manufacturer-provided device records.  
- The Fog Node (FN) hosts a Trusted Execution Environment (TEE) capable of remote attestation.  

Step 1: Device → Attribute Authority (Registration Request)

1. The IoT device prepares a registration request containing:
- device_ID
- manufacturer_meta (manufacturer-issued credential or metadata)
2. The device encrypts the request using the AA’s public key:  
Enc_pkAA(device_ID, manufacturer_meta)

3. The encrypted request is sent to the closest Attribute Authority.

Step 2: Attribute Authority Verification and Attribute Assignment

1. Upon receiving the request, the AA:

- Verifies the authenticity and integrity of device_ID using manufacturer-provided records.
- Confirms that the device is produced by a trusted manufacturer.

2. The AA retrieves the corresponding:

- Subject attributes
- Object attributes (if applicable)  
from the Device–Attribute Mapping Table.

3. The AA determines the applicable policies associated with these attributes.

4. The AA assigns a unique system-level identifier UID to the device.

5. The AA constructs a message:  
m = (UID, attributes, P[])

6. The AA signs the message and encrypts it using the public key of an attested TEE at the fog node:  
Enc_pkTEE(m, Sign_AA(m))

7. The encrypted message is sent to the fog node hosting the TEE.

Step 3: TEE Processing at Fog Node

1. The TEE decrypts the received message using its private key.

2. The TEE verifies the AA’s signature using the AA public key.

3. The TEE:  
- Seals attributes and policies references using TEE-provided sealing keys.
- Stores the sealed data outside the enclave, bound to the device UID.

4. The TEE generates a registration acknowledgment message:  
m' = (UID, FN_ID)

5. The TEE signs the acknowledgment to produce the response:  
r = Sign_TEE(m')

6. The response r is:  
- Returned to the IoT device
- Included in a blockchain transaction as immutable evidence of successful registration

Registration Outcome
- The IoT device is assigned a unique UID.
- Verified attributes and policy references are securely stored and bound to the device.
- The blockchain records the registration event without exposing sensitive attribute information.


#### Backup  

Attributes and access policies are sealed by the fog node’s TEE and stored outside the enclave on the fog node.(Attributes & policies encrypted using AES-GCM(K_store), symmetric key enryption is faster than public key encryption) To prevent data loss due to fog node failures, the fog node backs up authorization state to the cloud-based DDS after first registeration or state update.

During backup, the TEE load and unseal the local data, attaches metadata including owner identity, version number, and timestamp, and transmits the data to the cloud together with a TEE-generated signature. Upon receipt, the cloud verifies the signature and freshness before storing the backup data. This design enables efficient recovery while guarntee integrity and authenticity of authorization information.


#### Update, Revoke
smart conntract:  
AttributeManageContract has updateAttribute(), revokeAttribute();   
PolicyManageContract has updatePolicy(), revokePolicy()

Update  
When an attribute update is required, the responsible Attribute Authority (AA) generates an updated attribute set for the target device, identified by its UID. The updated attributes are encapsulated in an update message and signed by the AA to ensure authenticity and integrity. This message is then encrypted using the public key of the fog node’s TEE that manages the corresponding device domain.  
Upon receiving the update message, the TEE verifies the AA’s signature and decrypts the content inside the enclave. The TEE assigns a new attribute version identifier and reseals the updated attributes before storing them outside the enclave in encrypted form. Old attribute versions are retained for a configurable period to support auditability and rollback if necessary.  
To maintain global consistency, the fog node submits an attribute update transaction to the blockchain. The transaction contains only non-sensitive metadata, such as (UID, attribute_version, timestamp, Sign_TEE). The blockchain smart contract records this metadata immutably, enabling other system components to verify the freshness of attributes without accessing their contents.


Revoke  
revocation is necessary when a device becomes compromised, decommissioned, or no longer authorized to perform certain actions. In the proposed system, attribute revocation is enforced through version invalidation rather than physical deletion.  
When revocation is triggered, the AA issues a signed revocation message specifying the target UID and the attributes to be revoked. The message is securely transmitted to the relevant fog node’s TEE. After verifying the revocation request, the TEE marks the affected attribute versions as invalid and updates the local attribute status.  
A revocation transaction containing (UID, revoked_attribute_ids, timestamp, Sign_TEE) is then submitted to the blockchain. During authorization, the TEE checks the on-chain revocation status to ensure that revoked attributes are not used in policy evaluation. This approach prevents the reuse of stale or compromised attributes while maintaining an auditable record of revocation events.  


---
#### Authorization


AccessRequestRegistry Contract (ARRC)

Step 1: IoT → Blockchain (Access Request Transaction)  
The IoT device submits an access request as a transaction:  
Tx_req = (subject_ID, object_ID, action)

This transaction:
- Triggers the access request
- Is recorded for traceability
- Does not perform authorization

Step 2: Smart Contract (Request Validation + Event Emission)

Upon receiving Tx_req, the smart contract:
1. Verifies:
- subject_ID is registered
- object_ID is registered

2. fetch the responsible fog node FN_id of object from IoT–Fog mapping table.

3. Records request metadata (timestamp, request_ID)

4. Emits an AccessRequestEvent:  
Event(request_ID, subject_ID, object_ID, action, FN_id)  
FN_id helps direct the request to the correct fog node.

Step 3: Fog nodes listen
- Fog nodes listen to events
- Only fog nodes with FN-id respond
- Sends (subject_ID, object_ID, action) into TEE.

Step 4: Inside TEE

1. according to id, load subject attributes and object attributes from fog's normal space and unseal them.
3. retrieves matching policies according to attributes.
4. Executes authorization logic inside the TEE.
5. output access result res, TEE signature on this res.

Step 5: 
1. The fog node submits a transaction:
Tx = (request_ID, res, timestamp, Sign_TEE)
2. returns the access results to both the subject and object.


Upon receiving an access request, the fog node first authenticates the requesting IoT device by verifying its digital signature and checks request freshness using timestamps or nonces to prevent replay attacks. The fog node then verifies the existence and validity of the requested resource and its associated access policy before proceeding. Once these preliminary checks are completed, the fog node forwards the encrypted subject attributes and resource access policy to the Trusted Execution Environment (TEE) for authorization.

Within the enclave, the TEE securely decrypts the protected attributes and policies using enclave-bound keys and evaluates the access request according to predefined Attribute-Based Access Control (ABAC) rules. Because all sensitive authorization logic and data processing are confined within the enclave, neither a compromised fog operating system nor malicious insiders can manipulate attributes, alter policies, or influence authorization outcomes. The resulting access decision is then returned to the fog node for enforcement, enabling secure, fine-grained, and tamper-resistant authorization at the network edge

## Experiment

peformance evaluation: 
1. Registration performanc   
- Registration latency: Time from IoT registration request → on-chain confirmation
- TEE processing time: Attribute verification, sealing, attestation overhead
2. Authorization performance
- End-to-end authorization delay: IoT request → access decision
- TEE execution time: Attribute loading + policy evaluation
3. Throughput and Scalability
- number of IoT vs Authorization delay.
- transactions per second.
4. blockchain overhead: gas

### A.Setup


| Component           | Configuration (Example)         | Purpose                     |
| ------------------- | ------------------------------- | --------------------------- |
| IoT devices         | Raspberry Pi 4 / emulator       | Generate access requests    |
| Fog node            | Intel SGX-enabled server / DCAP | Host TEE + blockchain peer  |
| Blockchain peers    | 3–7 VMs                         | Consensus & ledger          |
| Attribute Authority | VM                              | Attribute management        |
| DDS / Cloud         | VM                              | Backup & cross-domain fetch |



#### Comparison:
| Baseline            | Weakness                      |
| ------------------- | ----------------------------- |
| Fully on-chain ABAC | High latency, privacy leakage |
| Fog-only ABAC       | No auditability               |
| Centralized AAA     | Single point of failure       |

gas comparison with ABAC only


#### _example (BTAA)_

In order to be able to document the actual time overhead, we conduct simulation experiments. For the simulation experiments in BTAA, we simulate the cryptographic operations in the authentication protocol and the user’s interaction with the blockchain, respectively.

For the cryptographic part of our protocol, we simulate it with the setting as follows. We simulate the AN and the ON in a laptop with an Intel Core i7-9750H CPU @ 2.60 GHz and 16.0 GB of RAM which is installed windows 10 Professional 64-bit operating system. We use Intel SGX as the TEE in our experiments. Our version of Intel SGX SDK and Intel SGX PSW for is v2.10.100.2. We use C and the PBC library to complete the encryption and signature algorithm in our protocol, which allows the computation of bilinear pairs. The version of pbc library we use in our experiments is pbc-0.5.14. Since pbc is not a trusted library for intel SGX, which means it is not possible to call functions belong to the pbc library directly in the enclave, so we use the ECALL/OCALL interface to call external functions for our experiments. We chose the curve in type F parameter provided by the pbc library for our experiments which can provide the asymmetric bilinear pairs.

For the part that interacts with the blockchain, we introduce the private blockchain into BTAA as a trusted distributed ledger for sharing domain-specific data. Our experimental environment is in a VMware workstation virtual machine with Ubuntu 20.04.2.0 LTS. We use geth to build an ethereum private chain as the public ledger for the experiment. The average transaction fee/cost is 23464 gas. The throughput of blockchain is 70 tps and the latency is 2.2 s. The blockNumber is 1577. The average block size is 801 byte and the average block time is 3.87 s. The reward per block is 0.293 ether and the hash rate is 33871.

## Disussion & Limitations
1. physical attacks against TEE
2. 